In [138]:
import requests
import time
import datetime as dt
import pandas as pd

import hashlib
import hmac
import base64

import json

from pprint import pprint 

from pandas.io.json import json_normalize

import numpy
import logging

In [139]:
# Current time stamp
timestamp = int(time.time())

In [140]:
logging.basicConfig(level=logging.INFO, 
                    format=' %(asctime)s - %(levelname)s - %(message)s')

In [141]:
# Date
today_date = datetime.date.today() + datetime.timedelta(days=0)
yesterday_date = datetime.date.today() + datetime.timedelta(days=-1)
seven_days_before_date = datetime.date.today() + datetime.timedelta(days=-7)

In [142]:
# Shop Parameter
partner_id = 12155
shopid = 23070969
shop_key = 'de5e924b8ed680bc9b22a6c402058154340333ff05641a38069f2856a2f3e24e'

In [143]:
# get_auth_signature
def get_auth_signature(url, payload):
    # API Key
    shopee_secret = shop_key
    # Post Message
    post_message = url + '|' + json.dumps(payload)
    # HMAC-SHA256
    message = bytes(post_message, 'utf-8')
    secret = bytes(shopee_secret, 'utf-8')
    hash = hmac.new(secret, message, hashlib.sha256)
    auth_signature = hash.hexdigest()
    return auth_signature

In [144]:
# URL
url = 'https://partner.shopeemobile.com/api/v1/items/get'

In [145]:
# get all item
max_page = 200
pagination_entries_per_page = 100
i = 0

try:
    while i <= max_page:
        # start entry
        start_entry = i * pagination_entries_per_page

        # Parameter
        pagination_offset = start_entry

        # payload
        payload = {"partner_id":partner_id,
                   "shopid":shopid,
                   "timestamp":timestamp,
                   "pagination_offset":pagination_offset,
                   "pagination_entries_per_page":pagination_entries_per_page}
        # Auth Signature
        Auth_signature = get_auth_signature(url, payload)
        # HTML Header
        headers = {"Host": "partner.shopeemobile.com",
                    "Content-Type": "application/json",
                    "Content-Length": "89",
                    "Authorization": Auth_signature}
        # HTML Body
        payload = json.dumps(payload)
        # Post Request
        r = requests.post(url, data=payload, headers=headers)
        # 打印原始Json
        parsed = json.loads(r.text)
        # 漂亮打印Json
        item_list = json_normalize(parsed['items'])
        # 排除update_time
        selected_column = ['item_id', 'shopid', 'status']
        item_list = item_list[selected_column]

        # 合并
        if i == 0:
            all_item_list = item_list
        else:
            all_item_list = all_item_list.append(item_list)

        i += 1
        logging.info('Retrieving item from page ' + str(i))
except:
    pass

# 去重
logging.info('Remove duplicate records.')
all_item_list = all_item_list.drop_duplicates('item_id').reset_index()
logging.info('Retrieved item list.')
print(all_item_list.head())

 2018-05-12 15:50:33,928 - INFO - Starting new HTTPS connection (1): partner.shopeemobile.com
 2018-05-12 15:50:34,463 - INFO - Retrieving item from page 1
 2018-05-12 15:50:34,466 - INFO - Starting new HTTPS connection (1): partner.shopeemobile.com
 2018-05-12 15:50:34,697 - INFO - Retrieving item from page 2
 2018-05-12 15:50:34,700 - INFO - Starting new HTTPS connection (1): partner.shopeemobile.com
 2018-05-12 15:50:34,996 - INFO - Remove duplicate records.
 2018-05-12 15:50:34,999 - INFO - Retrieved item list.


   index     item_id    shopid  status
0      0  1121446661  23070969  NORMAL
1      1  1121455576  23070969  NORMAL
2      2  1121423066  23070969  NORMAL
3      3   725349781  23070969  NORMAL
4      4   725351767  23070969  NORMAL


In [146]:
# get item detail
item_detail_url = 'https://partner.shopeemobile.com/api/v1/item/get'

In [147]:
item_detail_list = pd.DataFrame()

for key, item in all_item_list[:3].iterrows():
    try:
        item_id = item['item_id']
        logging.info('Now retrieving ' + str(item_id) + ' detail.')
        # payload
        payload = {"partner_id":partner_id,
                   "shopid":shopid,
                   "timestamp":timestamp,
                   "item_id": item_id}
        # Auth Signature
        Auth_signature = get_auth_signature(item_detail_url, payload)
        # HTML Header
        headers = {"Host": "partner.shopeemobile.com",
                    "Content-Type": "application/json",
                    "Content-Length": "89",
                    "Authorization": Auth_signature}
        # HTML Body
        payload = json.dumps(payload)
        # Post Request
        r = requests.post(item_detail_url, data=payload, headers=headers)
        # 打印原始Json
        parsed = json.loads(r.text)
        # 漂亮打印Json
        item_detail = json_normalize(parsed['item'])
        item_detail_list = item_detail_list.append(item_detail)
        
        # print(item_detail)
        logging.info('Retrieved ' + str(item_id) + ' detail.')
    except:
        pass
    
# convert timestamp to date
item_detail_list['create_time'] = pd.to_datetime(item_detail_list['create_time'], unit='s').dt.date
item_detail_list['update_time'] = pd.to_datetime(item_detail_list['update_time'], unit='s').dt.date

 2018-05-12 15:50:35,034 - INFO - Now retrieving 1121446661 detail.
 2018-05-12 15:50:35,037 - INFO - Starting new HTTPS connection (1): partner.shopeemobile.com
 2018-05-12 15:50:35,381 - INFO - Retrieved 1121446661 detail.
 2018-05-12 15:50:35,382 - INFO - Now retrieving 1121455576 detail.
 2018-05-12 15:50:35,385 - INFO - Starting new HTTPS connection (1): partner.shopeemobile.com
 2018-05-12 15:50:35,832 - INFO - Retrieved 1121455576 detail.
 2018-05-12 15:50:35,833 - INFO - Now retrieving 1121423066 detail.
 2018-05-12 15:50:35,836 - INFO - Starting new HTTPS connection (1): partner.shopeemobile.com
 2018-05-12 15:50:36,151 - INFO - Retrieved 1121423066 detail.


In [148]:
item_detail_list

,attributes,category_id,cmt_count,create_time,currency,days_to_ship,description,has_variation,images,item_id,...,rating_star,sales,shopid,status,stock,update_time,variations,views,weight,wholesales
0,"[{'attribute_type': 'STRING_TYPE', 'attribute_...",5804,0,2018-05-02,TWD,3,Fashion Patented High Quality Anti Slip Crash ...,True,[http://f.shopee.tw/file/a7a43d76b0361451988df...,1121446661,...,0.0,0,23070969,NORMAL,5000,2018-05-04,"[{'update_time': 1525277454, 'price': 129.0, '...",0,0.1,[]
0,"[{'attribute_type': 'STRING_TYPE', 'attribute_...",5804,0,2018-05-02,TWD,3,Lovely Cartoon Anti Slip Crash Proof Phone Cas...,True,[http://f.shopee.tw/file/43dd056d8d0a1fb574ce1...,1121455576,...,0.0,0,23070969,NORMAL,8000,2018-05-02,"[{'update_time': 1525277452, 'price': 129.0, '...",0,0.1,[]
0,"[{'attribute_type': 'STRING_TYPE', 'attribute_...",5804,0,2018-05-02,TWD,3,Fashion Patented High Quality Anti Slip Crash ...,True,[http://f.shopee.tw/file/e54a6fd53d6872355c7d3...,1121423066,...,0.0,0,23070969,NORMAL,9000,2018-05-02,"[{'update_time': 1525276430, 'price': 199.0, '...",2,0.1,[]


In [149]:
# item_detail_list.to_csv("D://item_detail_list.csv", sep=',')